In [13]:
morzesabc_dict = {'A': '.-', 'B': '-...', 'C': '-.-.', 
                    'D': '-..', 'E': '.', 'F': '..-.', 
                    'G': '--.', 'H': '....','I': '..', 
                    'J': '.---', 'K': '-.-','L': '.-..',
                    'M': '--', 'N': '-.','O': '---', 
                    'P': '.--.','Q': '--.-','R': '.-.', 
                    'S': '...', 'T': '-','U': '..-', 
                    'V': '...-', 'W': '.--','X': '-..-',
                    'Y': '-.--', 'Z': '--..',
                    '1': '.----','2': '..---', '3': '...--',
                    '4': '....-', '5': '.....', '6': '-....',
                    '7': '--...', '8': '---..', '9': '----.',
                    '0': '-----', ', ': '--..--', '.': '.-.-.-',
                    '?': '..--..', '/': '-..-.', '-': '-....-',
                    '(': '-.--.', ')': '-.--.-'}


def encrypted(message):

    encrypted_text = ""
    for letters in message:
        if letters != " ":
            encrypted_text = encrypted_text + \
                morzesabc_dict.get(letters) + " "

        else:
            encrypted_text += " "

    print(encrypted_text)


text = input("Text:")
print('Encrypted:')
encrypted(text)


Encrypted text:


TypeError: can only concatenate str (not "NoneType") to str

In [1]:
import tkinter
import pytube

root = tkinter.Tk()
root.geometry("350x325")
root.title("YouTube Downloader")
root.resizable(0,0)
root.wm_minsize(0,0)


myVar = tkinter.StringVar()
myVar.set("YouTube Link:")
tkinter.Label(root, textvariable=myVar, width=50).pack(pady=5)

link = tkinter.StringVar()

def download():
   
        myVar.set("")
        root.update()
        pytube.YouTube(link.get()).streams.first().download(skip_existing=True, output_path="./")
        w = TextWrapper()
        w.width = 50
        title_wrapped = "\n".join(w.wrap(str(pytube.YouTube(link.get()).title)))
        myVar.set("Great success".format(title_wrapped))
   

tkinter.Entry(root, textvariable=link, width=40).pack(pady=10)
tkinter.Button(root, text=f"Download!", command=download, fg="green", bg="white").pack()


root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/urllib/request.py", line 1346, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/http/client.py", line 1279, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/http/client.py", line 1325, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/http/client.py", line 1274, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/http/client.py", line 1034, in _send_output
    self.send(msg)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/http/client.py",

In [15]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os

# account credentials
username = "bulpau@gamil.com"
password = "Raudona69!"


def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)

# number of top emails to fetch
N = 3

# create an IMAP4 class with SSL, use your email provider's IMAP server
imap = imaplib.IMAP4_SSL("imap.gmail.com")
# authenticate
imap.login(username, password)

# select a mailbox (in this case, the inbox mailbox)
# use imap.list() to get the list of mailboxes
status, messages = imap.select("INBOX")

# total number of emails
messages = int(messages[0])

for i in range(messages, messages-N, -1):
    # fetch the email message by ID
    res, msg = imap.fetch(str(i), "(RFC822)")
    for response in msg:
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            # decode the email subject
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                # if it's a bytes, decode to str
                subject = subject.decode(encoding)
            # decode email sender
            From, encoding = decode_header(msg.get("From"))[0]
            if isinstance(From, bytes):
                From = From.decode(encoding)
            print("Subject:", subject)
            print("From:", From)
            # if the email message is multipart
            if msg.is_multipart():
                # iterate over email parts
                for part in msg.walk():
                    # extract content type of email
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # get the email body
                        body = part.get_payload(decode=True).decode()
                    except:
                        pass
                    if content_type == "text/plain" and "attachment" not in content_disposition:
                        # print text/plain emails and skip attachments
                        print(body)
                    elif "attachment" in content_disposition:
                        # download attachment
                        filename = part.get_filename()
                        if filename:
                            folder_name = clean(subject)
                            if not os.path.isdir(folder_name):
                                # make a folder for this email (named after the subject)
                                os.mkdir(folder_name)
                            filepath = os.path.join(folder_name, filename)
                            # download attachment and save it
                            open(filepath, "wb").write(part.get_payload(decode=True))
            else:
                # extract content type of email
                content_type = msg.get_content_type()
                # get the email body
                body = msg.get_payload(decode=True).decode()
                if content_type == "text/plain":
                    # print only text email parts
                    print(body)
            if content_type == "text/html":
                # if it's HTML, create a new HTML file and open it in browser
                folder_name = clean(subject)
                if not os.path.isdir(folder_name):
                    # make a folder for this email (named after the subject)
                    os.mkdir(folder_name)
                filename = "index.html"
                filepath = os.path.join(folder_name, filename)
                # write the file
                open(filepath, "w").write(body)
                # open in the default browser
                webbrowser.open(filepath)
            print("="*100)
# close the connection and logout
imap.close()
imap.logout()

error: b'[AUTHENTICATIONFAILED] Invalid credentials (Failure)'